In [6]:
import pandas as pd
import os

# --- KONFIGURACJA ---
LOKALIZACJA_FILE = 'lokalizacja.csv'
SALEFLATS_FILE = 'saleflats_2024_2025.csv'

# --- FUNKCJA DO DRUKOWANIA NAGŁÓWKÓW ---
def print_header(title):
    print("\n" + "="*80)
    print(f" {title} ".center(80, "="))
    print("="*80)

# --- GŁÓWNY SKRYPT ANALIZY ---
try:
    # ==============================================================================
    # 1. ANALIZA PLIKU `lokalizacja.csv` (POPRAWIONA WERSJA)
    # ==============================================================================
    print_header(f"Analiza pliku: {LOKALIZACJA_FILE}")
    
    LOK_COLUMNS = ['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName']
    df_lok = pd.read_csv(
        LOKALIZACJA_FILE, 
        header=None, 
        names=LOK_COLUMNS,
        na_values='\\N',
        low_memory=False
    )

    print("\n[INFO] Podstawowe informacje o strukturze (df.info()):")
    df_lok.info()

    print(f"\n[INFO] Wymiary tabeli (kształt): {df_lok.shape}")
    print(f"\n[INFO] Liczba unikalnych ID: {df_lok['Id'].nunique()}")

    print("\n[INFO] Sprawdzenie brakujących wartości (isnull().sum()):")
    print(df_lok.isnull().sum())

    print("\n[INFO] Pierwsze 5 wierszy (df.head()):")
    display(df_lok.head())

    print("\n[INFO] Unikalne wartości w kolumnie 'AdditionalName' (typy lokalizacji):")
    print(df_lok['AdditionalName'].value_counts())


    # ==============================================================================
    # 2. ANALIZA PLIKU `saleflats_2024_2025.csv` (WERSJA Z NOWYM SILNIKIEM)
    # ==============================================================================
    print_header(f"Analiza pliku: {SALEFLATS_FILE}")
    
    # Używamy silnika 'python', który jest bardziej elastyczny dla "brudnych" plików CSV.
    # Może działać wolniej, ale powinien poradzić sobie z problemem.
    print("\n[UWAGA] Wczytuję plik saleflats przy użyciu silnika 'python' z powodu błędu tokenizacji. Może to chwilę potrwać...")
    df_flats = pd.read_csv(SALEFLATS_FILE, engine='python')

    print("\n[INFO] Podstawowe informacje o strukturze (df.info()):")
    df_flats.info()

    print(f"\n[INFO] Wymiary tabeli (kształt): {df_flats.shape}")
    print("\n[INFO] Lista wszystkich znalezionych kolumn:")
    print(df_flats.columns.tolist())

    print("\n[INFO] Pierwsze 5 wierszy (df.head()):")
    display(df_flats.head())
    
    # Wybierzmy kolumny, które wyglądają na numeryczne, aby uniknąć błędów
    numeric_cols = df_flats.select_dtypes(include=np.number).columns.tolist()
    print("\n[INFO] Statystyki tylko dla kolumn numerycznych (df.describe()):")
    if numeric_cols:
        display(df_flats[numeric_cols].describe())
    else:
        print("Nie znaleziono kolumn numerycznych do opisu.")

    print("\n[INFO] Sprawdzenie brakujących wartości (isnull().sum()):")
    print(df_flats.isnull().sum())
    
    # Sprawdzamy kluczowe kolumny z Twojego zapytania, jeśli istnieją
    key_columns_to_check = ['BuldingType', 'locationPath']
    for col in key_columns_to_check:
        if col in df_flats.columns:
            print(f"\n[INFO] Unikalne wartości w kolumnie '{col}':")
            print(df_flats[col].value_counts(dropna=False).head(10)) # Pokaż 10 najczęstszych
        else:
            print(f"\n[UWAGA] Kolumna '{col}' nie została znaleziona w pliku.")

    # --- Analiza kluczowej kolumny 'locationPath' ---
    if 'locationPath' in df_flats.columns:
        print("\n--- Szczegółowa analiza kolumny 'locationPath' ---")
        
        if df_flats['locationPath'].isnull().any():
            print(f"\n[UWAGA] Znaleziono {df_flats['locationPath'].isnull().sum()} brakujących wartości w 'locationPath'!")
            df_flats.dropna(subset=['locationPath'], inplace=True)
            print("Wiersze z brakującym locationPath zostały usunięte na potrzeby analizy.")
        
        path_lengths = df_flats['locationPath'].astype(str).str.split(',').str.len()
        print("\n[INFO] Liczba części w 'locationPath' (oczekiwano 7):")
        print(path_lengths.value_counts())

        try:
            loc_path_cols = [f'loc_{i}' for i in range(1, 8)]
            temp_loc_df = df_flats['locationPath'].str.split(',', expand=True)
            # Upewniamy się, że mamy 7 kolumn
            if temp_loc_df.shape[1] == 7:
                temp_loc_df.columns = loc_path_cols
                temp_loc_df = temp_loc_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

                missing_district_count = (temp_loc_df['loc_5'] == 0).sum()
                total_count = len(df_flats)
                print(f"\n[INFO] Liczba ogłoszeń z brakującą DZIELNICĄ (ID = 0): {missing_district_count} z {total_count} ({missing_district_count/total_count:.2%})")
                
                missing_street_count = (temp_loc_df['loc_7'] == 0).sum()
                print(f"[INFO] Liczba ogłoszeń z brakującą ULICĄ (ID = 0): {missing_street_count} z {total_count} ({missing_street_count/total_count:.2%})")

                city_ids = temp_loc_df['loc_4'].unique()
                print(f"\n[INFO] Znaleziono {len(city_ids)} unikalnych ID miast (kolumna 4).")
                
                city_id_to_name = df_lok[df_lok['Id'].isin(city_ids)].set_index('Id')['Name'].to_dict()
                print("[INFO] Rozkład ogłoszeń wg miast:")
                print(temp_loc_df['loc_4'].map(city_id_to_name).value_counts())
            else:
                print(f"\n[BŁĄD] 'locationPath' nie dzieli się na 7 kolumn. Znaleziono {temp_loc_df.shape[1]} kolumn.")

        except Exception as e:
            print(f"\n[BŁĄD] Nie można było przeanalizować 'locationPath'. Błąd: {e}")


except FileNotFoundError as e:
    print(f"BŁĄD: Nie znaleziono pliku! Upewnij się, że plik '{e.filename}' znajduje się w tym samym folderze co notatnik.")
except Exception as e:
    print(f"Wystąpił nieoczekiwany błąd: {e}")

print("\n" + "="*80)
print(" Analiza zakończona. Skopiuj całą powyższą treść i wklej w odpowiedzi. ".center(80, "="))
print("="*80)


======================== Analiza pliku: lokalizacja.csv ========================

[INFO] Podstawowe informacje o strukturze (df.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398832 entries, 0 to 398831
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Id              398832 non-null  int64  
 1   ParentId        398816 non-null  float64
 2   Name            398832 non-null  object 
 3   AdditionalName  398832 non-null  object 
 4   FullName        398832 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 15.2+ MB

[INFO] Wymiary tabeli (kształt): (398832, 5)

[INFO] Liczba unikalnych ID: 398832

[INFO] Sprawdzenie brakujących wartości (isnull().sum()):
Id                 0
ParentId          16
Name               0
AdditionalName     0
FullName           0
dtype: int64

[INFO] Pierwsze 5 wierszy (df.head()):


,Id,ParentId,Name,AdditionalName,FullName
0,1,NaN,Świętokrzyskie,Województwo,Świętokrzyskie
1,2,NaN,Śląskie,Województwo,Śląskie
2,3,NaN,Podlaskie,Województwo,Podlaskie
3,4,NaN,Opolskie,Województwo,Opolskie
4,5,NaN,Zachodniopomorskie,Województwo,Zachodniopomorskie



[INFO] Unikalne wartości w kolumnie 'AdditionalName' (typy lokalizacji):
AdditionalName
Ul.                                    280788
Wieś                                    43160
Część                                   35707
Część miasta                             7032
Osada                                    5605
Przysiółek                               4760
Rondo                                    3228
Kolonia                                  3148
Pl.                                      2872
Osada Leśna                              2113
Osiedle                                  1731
Gmina wiejska                            1498
Skwer                                    1379
Inne                                     1059
Al.                                       927
Miasto                                    913
Park                                      706
Gmina miejsko-wiejska                     677
Obszar wiejski                            677
Powiat                               

In [7]:
import pandas as pd

# --- KONFIGURACJA ---
SALEFLATS_FILE = 'saleflats_2024_2025.csv'

print("="*80)
print("Odczytuję tylko nagłówek (pierwszy wiersz) z pliku saleflats...")
print("="*80)

try:
    # Wczytujemy tylko nagłówek (nrows=0) lub pierwszy wiersz (nrows=1), co jest bezpieczną operacją.
    # Używamy silnika python, który jest bardziej elastyczny.
    header_df = pd.read_csv(SALEFLATS_FILE, nrows=1, engine='python')
    
    column_names = header_df.columns.tolist()
    
    print("[SUKCES] Udało się odczytać nazwy kolumn.")
    print(f"Plik zawiera {len(column_names)} kolumn.")
    print("\nOto pełna lista nazw kolumn (wraz z ich numerem porządkowym):")
    
    for i, name in enumerate(column_names):
        print(f"{i:02d}: '{name}'")
        
    print("\n" + "="*80)
    print("PROSZĘ O TWOJĄ POMOC:")
    print("1. Przejrzyj powyższą listę.")
    print("2. Zidentyfikuj, które nazwy kolumn odpowiadają wymaganym cechom:")
    print("   - Area (Powierzchnia)")
    print("   - Price (Cena)")
    print("   - NumberOfRooms (Liczba pokoi)")
    print("   - Floor (Piętro)")
    print("   - Floors (Liczba pięter w budynku)")
    print("   - BuiltYear (Rok budowy)")
    print("   - BuldingType (Typ budynku)")
    print("   - Title (Tytuł ogłoszenia)")
    print("   - Description (Opis)")
    print("   - locationPath")
    print("\n3. Wklej mi w odpowiedzi listę tych prawidłowych nazw kolumn. Na przykład:")
    print("   Price: 'price'")
    print("   Area: 'sqm'")
    print("   Description: 'opis_dlugi'")
    print("   itd.")
    print("="*80)

except Exception as e:
    print(f"\nWystąpił błąd nawet przy próbie odczytu nagłówka: {e}")

Odczytuję tylko nagłówek (pierwszy wiersz) z pliku saleflats...
[SUKCES] Udało się odczytać nazwy kolumn.
Plik zawiera 53 kolumn.

Oto pełna lista nazw kolumn (wraz z ich numerem porządkowym):
00: '88'
01: 'NULL'
02: '14'
03: 'Mieszkanie trzypokojowe na sprzedaż'
04: 'Mieszkanie o powierzchni 73m2 znajduje się na 3 piętrze bloku z cegły z 2005 roku, w doskonałej lokalizacji osiedla Bojary, zaledwie 2 minuty od Pałacu Branickich, co sprawia, że jest to ścisłe centrum miasta. Mieszkanie zostało urządzone w swojskim klimacie, co czyni je wyjątkowym i wyróżnia się spośród innych dostępnych na rynku. Wejście do mieszkania jest wygodne i szerokie, a dodatkowo została zamontowana szafa wnękowa na okrycia wierzchnie, co zapewnia praktyczne rozwiązanie na przechowywanie ubrań i innych przedmiotów. Zabudowa kuchenna w mieszkaniu jest wyjątkowo zaakcentowana i wykonana z płytek z glazury, co nadaje jej wiejski klimat. Jest to unikalne rozwiązanie, które wyróżnia się spośród innych standardowych k